In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import math

In [2]:
import cv2

In [3]:
import random

In [4]:
import os

In [5]:
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
from tensorflow.keras import layers

In [7]:
import keras.backend as K

# Recolección de datos

## Imagens a blanco y negro y redimensionada

In [24]:
input_images_path = "../input/archivo2/trainA"
files_names = os.listdir(input_images_path)

In [39]:
i = 0
foto_blanco_negro = []
for i in range(11):
    for file_name in files_names:
        image_path = input_images_path + "/" + file_name
        img = cv2.imread(image_path, 0)
        #print(img.shape)
        #print(img.shape)
        img_res = cv2.resize(img, dsize = (320, 240), interpolation=cv2.INTER_CUBIC)
        foto_blanco_negro.append(img_res)
print(len(foto_blanco_negro))

## Foto con la homografia

In [40]:
i = 0
pts_src_total = []
pts_dst_total = []
etiquetas = []
fotos_homografia = []
for img_homography in foto_blanco_negro: 
    #image_path = input_images_path + "/" + file_name
    #img_homography = cv2.imread(image_path)
    x = random.randint(100,220)
    y = random.randint(100,140)
    x1 = x - 64
    y1 = y - 64
    x2 = x + 64
    y2 = y - 64
    x3 = x + 64
    y3 = y + 64
    x4 = x - 64
    y4 = y + 64
    pts_src = []
    pts_src.append((x1,y1))
    pts_src.append((x2,y2))
    pts_src.append((x3,y3))
    pts_src.append((x4,y4))
    pts_src = np.array(pts_src)
    pts_src_total.append(pts_src)
    ###############################
    
    p = 32
    px1 = random.randint(- p, p)
    px2 = random.randint(- p, p)
    px3 = random.randint(- p, p)
    px4 = random.randint(- p, p)
    py1 = random.randint(- p, p)
    py2 = random.randint(- p, p)
    py3 = random.randint(- p, p)
    py4 = random.randint(- p, p)
    
    salida = []
    salida.append(px1)
    salida.append(py1)
    salida.append(px2)
    salida.append(py2)
    salida.append(px3)
    salida.append(py3)
    salida.append(px4)
    salida.append(py4)
    etiquetas.append(salida)
    
    x_1 = x1 + px1
    y_1 = y1 + py1
    x_2 = x2 + px2
    y_2 = y2 + py2
    x_3 = x3 + px3
    y_3 = y3 + py3
    x_4 = x4 + px4
    y_4 = y4 + py4

    pts_dst = []
    pts_dst.append((x_1,y_1))
    pts_dst.append((x_2,y_2))
    pts_dst.append((x_3,y_3))
    pts_dst.append((x_4,y_4))
    pts_dst = np.array(pts_dst)
    pts_dst_total.append(pts_dst)
    
    h, status = cv2.findHomography(pts_src, pts_dst)
    im_out = cv2.warpPerspective(img_homography, h, (img_homography.shape[1],img_homography.shape[0]))
    fotos_homografia.append(im_out)
    i += 1
    #print(file_name)

## 2 fotografias para la red

In [41]:
i = 0
homo1 = []
for img in fotos_homografia:
    #image_path = input_images_path + "/" + file_name
    #img = cv2.imread(image_path)
    #cv2.imwrite("Homografia_1/Foto_h" + str(i) + ".png", img)
    #print(p)
    p = pts_src_total[i]
    p2 = pts_src_total[i]
    
    #print(p)
    coor_1 = p[0]    
    coor_2 = p[2]
    
    x = coor_1[0]
    x2 = coor_2[0]
    
    y = coor_1[1]
    y2 = coor_2[1]
    #x2 = p2[2]
    #y2 = p2[1]
    
    #print(x, x2)
    #print(y, y2)
    crop_img1 = img[y:y2, x:x2]
    homo1.append(crop_img1)
    #cv2.imwrite("Homografia_1/" + file_name, crop_img1)
    i += 1
    #print("---------------")

In [42]:
i = 0
homo2 = []
for img in fotos_homografia:
    #image_path = input_images_path + "/" + file_name
    #img = cv2.imread(image_path)
    #cv2.imwrite("Homografia_1/Foto_h" + str(i) + ".png", img)
    p = pts_src_total[i]
    p2 = pts_src_total[i]
    
    #print(p)
    coor_1 = p[0]    
    coor_2 = p[2]
    
    x = coor_1[0]
    x2 = coor_2[0]
    
    y = coor_1[1]
    y2 = coor_2[1]
    #x2 = p2[2]
    #y2 = p2[1]
    
    #print(x, x2)
    #print(y, y2)
    crop_img1 = img[y:y2, x:x2]
    homo2.append(crop_img1)
    #cv2.imwrite("Homografia_2/" + file_name, crop_img1)
    i += 1
    #print("---------------")

# Juntamos las fotos y las etiquetas

In [43]:
fotos_train = []
#f = open("fotos_canales", "w")
for i in range(len(homo1)):
    for j in range(len(homo2)):
        if i == j:
            foto = np.stack((homo1[i], homo2[j]), axis = 2, out = None)
            #foto = np.swapaxes(np.stack([fotos_homo1[i], fotos_homo2[j]]),0,2)
            #f.write(str(foto) + "\n")
            fotos_train.append(foto)
#f.close()

In [44]:
print(len(fotos_train))
print(len(etiquetas))

In [45]:
def euclidean_distance(y_true, y_pred):
    """ Implements the L2 (Euclidean) loss function
    
        Parameters:
        y_true (float): ground-truth training label
        y_pred (float): predicted training label from the CNN
        
        Returns:
        L2 (Euclidean) loss function
    """
    return K.sqrt(K.maximum(K.sum(K.square(y_pred - y_true), axis=-1,
                                  keepdims=True), K.epsilon()))

In [46]:
fotos_train2 = []
etiquetas_train2 = []
fotos_val = []
etiquetas_val = []
c = 0
for i in fotos_train:
    if c <= len(fotos_train)*0.8:
        fotos_train2.append(i)
    else:
        fotos_val.append(i)
    c += 1
c = 0
for i in etiquetas:
    if c <= len(fotos_train)*0.8:
        etiquetas_train2.append(i)
    else:
        etiquetas_val.append(i)
    c += 1

In [47]:
fotos_val = np.array(fotos_val) 
etiquetas_val = np.array(etiquetas_val)
etiquetas_train2 = np.array(etiquetas_train2)
fotos_train2 = np.array(fotos_train2)
print(len(fotos_train2))
print(len(etiquetas_train2))
print(len(fotos_val))
print(len(etiquetas_val))

In [48]:
fotos_train3 = fotos_train2/255
fotos_val2 = fotos_val/255
etiquetas_val = etiquetas_val/32
etiquetas_train2 = etiquetas_train2/32

# 3 modelos diferentes, entrenados con 1000, 5000 y 8000 fotos 

## Modelo con 1000 fotos para entrenamiento

In [20]:
modelo = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation = tf.nn.relu, input_shape = (128,128,2), padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    #tf.keras.layers.Conv2D(64, (3,3), activation = tf.nn.relu, padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    #tf.keras.layers.MaxPooling2D(pool_size = [2,2], strides = 2),
    tf.keras.layers.Conv2D(64, (3,3), activation = tf.nn.relu, padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    #tf.keras.layers.Conv2D(64, (3,3), activation = tf.nn.relu, padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    tf.keras.layers.MaxPooling2D(pool_size = [2,2], strides = 2),
    tf.keras.layers.Conv2D(128, (3,3), activation = tf.nn.relu, padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    tf.keras.layers.Conv2D(128, (3,3), activation = tf.nn.relu, padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    #tf.keras.layers.MaxPooling2D(pool_size = [2,2], strides = 2),
    #tf.keras.layers.Conv2D(128, (3,3), activation = tf.nn.relu, padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    #tf.keras.layers.Conv2D(128, (3,3), activation = tf.nn.relu, padding = "same"),
    tf.keras.layers.MaxPooling2D(pool_size = [2,2], strides = 2),
    tf.keras.layers.Dropout(0.5),
    #tf.reshape([-1, 128 * 128 * 2]),    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation = "relu"),
    tf.keras.layers.Dense(8, activation = "tanh")
])

In [21]:
modelo.compile(optimizer = "adam",
               loss= "mean_squared_error",
               metrics = ["accuracy"])

In [22]:
history = modelo.fit(fotos_train3, etiquetas_train2, epochs=20,batch_size = 16, 
                    steps_per_epoch = int(np.ceil(len(fotos_train3)/float(16))), 
                    validation_steps = int(np.ceil(len(fotos_val2)/float(16))),
                    validation_data=(fotos_val2, etiquetas_val))


In [23]:
loss = history.history["val_loss"]
epochs = range(20)
#plt.plot(epochs, loss)
accuracy = history.history["val_accuracy"]
#epochs = range(20)
#plt.plot(epochs, accuracy)
fig = plt.figure(figsize =(30, 5))
plt.subplot(121),plt.title("Loss"),plt.plot(epochs,loss)
plt.subplot(122),plt.title("Accuracy"),plt.plot(epochs,accuracy)
plt.show()

## Modelo con 5000 fotos para entrenamiento

In [35]:
modelo2 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation = tf.nn.relu, input_shape = (128,128,2), padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    #tf.keras.layers.Conv2D(64, (3,3), activation = tf.nn.relu, padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    #tf.keras.layers.MaxPooling2D(pool_size = [2,2], strides = 2),
    tf.keras.layers.Conv2D(64, (3,3), activation = tf.nn.relu, padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    #tf.keras.layers.Conv2D(64, (3,3), activation = tf.nn.relu, padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    tf.keras.layers.MaxPooling2D(pool_size = [2,2], strides = 2),
    tf.keras.layers.Conv2D(128, (3,3), activation = tf.nn.relu, padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    tf.keras.layers.Conv2D(128, (3,3), activation = tf.nn.relu, padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    #tf.keras.layers.MaxPooling2D(pool_size = [2,2], strides = 2),
    #tf.keras.layers.Conv2D(128, (3,3), activation = tf.nn.relu, padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    #tf.keras.layers.Conv2D(128, (3,3), activation = tf.nn.relu, padding = "same"),
    tf.keras.layers.MaxPooling2D(pool_size = [2,2], strides = 2),
    tf.keras.layers.Dropout(0.5),
    #tf.reshape([-1, 128 * 128 * 2]),    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation = "relu"),
    tf.keras.layers.Dense(8, activation = "tanh")
])

In [36]:
modelo2.compile(optimizer = "adam",
               loss= "mean_squared_error",
               metrics = ["accuracy"])

In [37]:
history2 = modelo2.fit(fotos_train3, etiquetas_train2, epochs=50,batch_size = 16, 
                    steps_per_epoch = int(np.ceil(len(fotos_train3)/float(16))), 
                    validation_steps = int(np.ceil(len(fotos_val2)/float(16))),
                    validation_data=(fotos_val2, etiquetas_val))

In [38]:
loss = history2.history["val_loss"]
epochs = range(50)
#plt.plot(epochs, loss)
accuracy = history2.history["val_accuracy"]
#epochs = range(20)
#plt.plot(epochs, accuracy)
fig = plt.figure(figsize =(30, 5))
plt.subplot(121),plt.title("Loss"),plt.plot(epochs,loss)
plt.subplot(122),plt.title("Accuracy"),plt.plot(epochs,accuracy)
plt.show()

## Modelo con 11000 fotos para entrenamiento

In [49]:
modelo3 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation = tf.nn.relu, input_shape = (128,128,2), padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    #tf.keras.layers.Conv2D(64, (3,3), activation = tf.nn.relu, padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    #tf.keras.layers.MaxPooling2D(pool_size = [2,2], strides = 2),
    tf.keras.layers.Conv2D(64, (3,3), activation = tf.nn.relu, padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    #tf.keras.layers.Conv2D(64, (3,3), activation = tf.nn.relu, padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    tf.keras.layers.MaxPooling2D(pool_size = [2,2], strides = 2),
    tf.keras.layers.Conv2D(128, (3,3), activation = tf.nn.relu, padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    tf.keras.layers.Conv2D(128, (3,3), activation = tf.nn.relu, padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    #tf.keras.layers.MaxPooling2D(pool_size = [2,2], strides = 2),
    #tf.keras.layers.Conv2D(128, (3,3), activation = tf.nn.relu, padding = "same"),
    #tf.keras.layers.BatchNormalization(axis = -1),
    #tf.keras.layers.Conv2D(128, (3,3), activation = tf.nn.relu, padding = "same"),
    tf.keras.layers.MaxPooling2D(pool_size = [2,2], strides = 2),
    tf.keras.layers.Dropout(0.5),
    #tf.reshape([-1, 128 * 128 * 2]),    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation = "relu"),
    tf.keras.layers.Dense(8, activation = "tanh")
])

In [50]:
modelo3.compile(optimizer = "adam",
               loss= "mean_squared_error",
               metrics = ["accuracy"])

In [51]:
history3 = modelo3.fit(fotos_train3, etiquetas_train2, epochs=30,batch_size = 16, 
                    steps_per_epoch = int(np.ceil(len(fotos_train3)/float(16))), 
                    validation_steps = int(np.ceil(len(fotos_val2)/float(16))),
                    validation_data=(fotos_val2, etiquetas_val))

In [52]:
loss = history3.history["val_loss"]
epochs = range(30)
#plt.plot(epochs, loss)
accuracy = history3.history["val_accuracy"]
#epochs = range(20)
#plt.plot(epochs, accuracy)
fig = plt.figure(figsize =(30, 5))
plt.subplot(121),plt.title("Loss"),plt.plot(epochs,loss)
plt.subplot(122),plt.title("Accuracy"),plt.plot(epochs,accuracy)
plt.show()

In [ ]:
suma = 0
for i in range(len(fotos_val2)):
    #print(fotos_val2[i].shape)
    valor = fotos_val2[i]
    valor = np.expand_dims(valor, axis = 0)
    y = modelo.predict(valor)
    #print(round(y[0][0],1), round(y[0][1],1), round(y[0][2],1), round(y[0][3],1), round(y[0][4],1), round(y[0][5],1), round(y[0][6],1),
    #     round(y[0][7],1))
    #print(round(etiquetas_val[i][0],1), round(etiquetas_val[i][1],1), round(etiquetas_val[i][2],1), 
    #      round(etiquetas_val[i][3],1), round(etiquetas_val[i][4],1),
    #      round(etiquetas_val[i][5],1), round(etiquetas_val[i][5],1), round(etiquetas_val[i][5],1))
    prom = (np.abs(y[0][0] - etiquetas_val[i][0]) + np.abs(y[0][1] - etiquetas_val[i][1]) + np.abs(y[0][2] - etiquetas_val[i][2]) +
          np.abs(y[0][3] - etiquetas_val[i][3]) + np.abs(y[0][4] - etiquetas_val[i][4]) + np.abs(y[0][5] - etiquetas_val[i][5]) +
          np.abs(y[0][6] - etiquetas_val[i][6]) + np.abs(y[0][7] - etiquetas_val[i][7])) / 8
    #print("Promedio de error ----> ", prom)
    suma += prom 
    #print("-"*50)
prom_total = suma / len(fotos_val2)
print("Promedio total ----->", prom_total)
    

In [122]:
foto_blanco_negro = []
path = "../input/coco-2017-dataset/coco2017/train2017/000000000133.jpg"
img = cv2.imread(path, 0)
img_res = cv2.resize(img, dsize = (320, 240), interpolation=cv2.INTER_CUBIC)
foto_blanco_negro.append(img_res)

In [123]:
i = 0
pts_src_total = []
pts_dst_total = []
etiquetas = []
fotos_homografia = []
for img_homography in foto_blanco_negro: 
    #image_path = input_images_path + "/" + file_name
    #img_homography = cv2.imread(image_path)
    x = random.randint(100,220)
    y = random.randint(100,140)
    x1 = x - 64
    y1 = y - 64
    x2 = x + 64
    y2 = y - 64
    x3 = x + 64
    y3 = y + 64
    x4 = x - 64
    y4 = y + 64
    pts_src = []
    pts_src.append((x1,y1))
    pts_src.append((x2,y2))
    pts_src.append((x3,y3))
    pts_src.append((x4,y4))
    pts_src = np.array(pts_src)
    pts_src_total.append(pts_src)
    ###############################
    
    p = 32
    px1 = random.randint(- p, p)
    px2 = random.randint(- p, p)
    px3 = random.randint(- p, p)
    px4 = random.randint(- p, p)
    py1 = random.randint(- p, p)
    py2 = random.randint(- p, p)
    py3 = random.randint(- p, p)
    py4 = random.randint(- p, p)
    
    salida = []
    salida.append(px1)
    salida.append(py1)
    salida.append(px2)
    salida.append(py2)
    salida.append(px3)
    salida.append(py3)
    salida.append(px4)
    salida.append(py4)
    etiquetas.append(salida)
    
    x_1 = x1 + px1
    y_1 = y1 + py1
    x_2 = x2 + px2
    y_2 = y2 + py2
    x_3 = x3 + px3
    y_3 = y3 + py3
    x_4 = x4 + px4
    y_4 = y4 + py4

    pts_dst = []
    pts_dst.append((x_1,y_1))
    pts_dst.append((x_2,y_2))
    pts_dst.append((x_3,y_3))
    pts_dst.append((x_4,y_4))
    pts_dst = np.array(pts_dst)
    pts_dst_total.append(pts_dst)
    
    h, status = cv2.findHomography(pts_src, pts_dst)
    im_out = cv2.warpPerspective(img_homography, h, (img_homography.shape[1],img_homography.shape[0]))
    fotos_homografia.append(im_out)
    i += 1
    #print(file_name)

In [124]:
i = 0
homo1 = []
for img in fotos_homografia:
    #image_path = input_images_path + "/" + file_name
    #img = cv2.imread(image_path)
    #cv2.imwrite("Homografia_1/Foto_h" + str(i) + ".png", img)
    #print(p)
    p = pts_src_total[i]
    p2 = pts_src_total[i]
    
    #print(p)
    coor_1 = p[0]    
    coor_2 = p[2]
    
    x = coor_1[0]
    x2 = coor_2[0]
    
    y = coor_1[1]
    y2 = coor_2[1]
    #x2 = p2[2]
    #y2 = p2[1]
    
    #print(x, x2)
    #print(y, y2)
    crop_img1 = img[y:y2, x:x2]
    homo1.append(crop_img1)
    #cv2.imwrite("Homografia_1/" + file_name, crop_img1)
    i += 1
    #print("---------------")

In [125]:
i = 0
homo2 = []
for img in fotos_homografia:
    #image_path = input_images_path + "/" + file_name
    #img = cv2.imread(image_path)
    #cv2.imwrite("Homografia_1/Foto_h" + str(i) + ".png", img)
    p = pts_src_total[i]
    p2 = pts_src_total[i]
    
    #print(p)
    coor_1 = p[0]    
    coor_2 = p[2]
    
    x = coor_1[0]
    x2 = coor_2[0]
    
    y = coor_1[1]
    y2 = coor_2[1]
    #x2 = p2[2]
    #y2 = p2[1]
    
    #print(x, x2)
    #print(y, y2)
    crop_img1 = img[y:y2, x:x2]
    homo2.append(crop_img1)
    #cv2.imwrite("Homografia_2/" + file_name, crop_img1)
    i += 1
    #print("---------------")

In [126]:
fotos_train = []
#f = open("fotos_canales", "w")
for i in range(len(homo1)):
    for j in range(len(homo2)):
        if i == j:
            foto = np.stack((homo1[i], homo2[j]), axis = 2, out = None)
            #foto = np.swapaxes(np.stack([fotos_homo1[i], fotos_homo2[j]]),0,2)
            #f.write(str(foto) + "\n")
            fotos_train.append(foto)
#f.close()
fotos_train = np.array(fotos_train)
fotos_train = fotos_train/255
print(fotos_train[0].shape)

In [127]:
x1 = pts_src[0]
x2 = pts_src[1]
x3 = pts_src[2]
x4 = pts_src[3]

val1 = x1[0]
val2 = x1[1]
val3 = x2[0]
val4 = x2[1]
val5 = x3[0]
val6 = x3[1]
val7 = x4[0]
val8 = x4[1]

In [128]:
valor = fotos_train[0]
valor = np.expand_dims(valor, axis = 0)

y = modelo.predict(valor)
prediccion = y * 32
prediccion = prediccion[0]
pts_dst_new = []

pts_dst_new.append((int(val1 + prediccion[0]),int(val2 + prediccion[1])))
pts_dst_new.append((int(val3 + prediccion[2]),int(val4 + prediccion[3])))
pts_dst_new.append((int(val5 + prediccion[4]),int(val6 + prediccion[5])))
pts_dst_new.append((int(val7 + prediccion[6]),int(val8 + prediccion[7])))

pts_dst_new = np.array(pts_dst_new)

###########################################

y2 = modelo2.predict(valor)
prediccion2 = y2 * 32
prediccion2 = prediccion2[0]
pts_dst_new2 = []

pts_dst_new2.append((int(val1 + prediccion2[0]),int(val2 + prediccion2[1])))
pts_dst_new2.append((int(val3 + prediccion2[2]),int(val4 + prediccion2[3])))
pts_dst_new2.append((int(val5 + prediccion2[4]),int(val6 + prediccion2[5])))
pts_dst_new2.append((int(val7 + prediccion2[6]),int(val8 + prediccion2[7])))

pts_dst_new2 = np.array(pts_dst_new2)

##########################################

y3 = modelo3.predict(valor)
prediccion3 = y3 * 32
prediccion3 = prediccion3[0]
pts_dst_new3 = []

pts_dst_new3.append((int(val1 + prediccion3[0]),int(val2 + prediccion3[1])))
pts_dst_new3.append((int(val3 + prediccion3[2]),int(val4 + prediccion3[3])))
pts_dst_new3.append((int(val5 + prediccion3[4]),int(val6 + prediccion3[5])))
pts_dst_new3.append((int(val7 + prediccion3[6]),int(val8 + prediccion3[7])))

pts_dst_new3 = np.array(pts_dst_new3)

In [139]:
img3 = cv2.imread(path)
img3 = cv2.resize(img3, dsize = (320, 240), interpolation=cv2.INTER_CUBIC)
h, status = cv2.findHomography(pts_src, pts_dst)
im_out = cv2.warpPerspective(img3, h, (img3.shape[1],img3.shape[0]))
#fotos_homografia.append(im_out)

In [140]:
img2 = cv2.imread(path)
img2 = cv2.resize(img2, dsize = (320, 240), interpolation=cv2.INTER_CUBIC)
color = (255, 0, 0)
color2 = (0,255,0)
thickness = 2
isClosed = True

cv2.polylines(img2, [pts_src], isClosed, color, thickness)
#cv2.polylines(img,[pts_dst], isClosed, color, thickness)
#cv2.polylines(img, [pts_dst_new], isClosed, color2, thickness)
#cv2.imwrite("res1.png", img)
#cv2.imwrite("res2.png", img2)

In [141]:
#img2 = cv2.imread("../input/corpusmanzanas/testA/n07740461_3211.jpg")

color = (255, 0, 0)
color2 = (0,255,0)
thickness = 2
isClosed = True

#cv2.polylines(img, [pts_src], isClosed, color, thickness)
cv2.polylines(im_out,[pts_dst], isClosed, color2, thickness)
#cv2.polylines(img, [pts_dst_new], isClosed, color2, thickness)
#cv2.imwrite("res2.png", img)
#cv2.imwrite("res2.png", img2)

In [142]:
#img = cv2.imread("res2.png")

color = (255, 0, 0)
color2 = (0,255,0)
color3 = (0,0,255)
thickness = 2
isClosed = True

#cv2.polylines(img, [pts_src], isClosed, color, thickness)
#cv2.polylines(img,[pts_dst], isClosed, color, thickness)
cv2.polylines(im_out, [pts_dst_new3], isClosed, color3, thickness)
#cv2.imwrite("res3.png", img)
#cv2.imwrite("res2.png", img2)

# Calculo con la misma foto y mismas correspondencias de homografia pero con las diferentes predicicones de los 3 modelos diferentes

# FOTO 1

## Para modelo 1 (1000 fotos)

In [110]:
#img = Image.open("res1.png")
#img2 = Image.open("res3.png")
print("PUNTOS REALES")
print(pts_dst)
print("PUNTOS OBTENIDOS DE LA RED")
print(pts_dst_new)
fig = plt.figure(figsize =(25, 15))
plt.subplot(121),plt.title("1 Cuadro"),plt.imshow(img2)
plt.subplot(122),plt.title("2 Cuadros"),plt.imshow(im_out)
plt.show()

## Para modelo 2 (5000 fotos)

In [115]:
#img = Image.open("res1.png")
#img2 = Image.open("res3.png")
print("PUNTOS REALES")
print(pts_dst)
print("PUNTOS OBTENIDOS DE LA RED")
print(pts_dst_new2)
fig = plt.figure(figsize =(25, 15))
plt.subplot(121),plt.title("1 Cuadro"),plt.imshow(img2)
plt.subplot(122),plt.title("2 Cuadros"),plt.imshow(im_out)
plt.show()

## Para modelo 3 (11000 fotos)

In [120]:
#img = Image.open("res1.png")
#img2 = Image.open("res3.png")
print("PUNTOS REALES")
print(pts_dst)
print("PUNTOS OBTENIDOS DE LA RED")
print(pts_dst_new3)
print("-"*50)
fig = plt.figure(figsize =(25, 15))
plt.subplot(121),plt.title("1 Cuadro"),plt.imshow(img2)
plt.subplot(122),plt.title("2 Cuadros"),plt.imshow(im_out)
plt.show()

# FOTO 2

## Para modelo 1 (1000 fotos)

In [88]:
#img = Image.open("res1.png")
#img2 = Image.open("res3.png")
print("PUNTOS REALES")
print(pts_dst)
print("PUNTOS OBTENIDOS DE LA RED")
print(pts_dst_new)
print("-"*50)
fig = plt.figure(figsize =(25, 15))
plt.subplot(121),plt.title("1 Cuadro"),plt.imshow(img2)
plt.subplot(122),plt.title("2 Cuadros"),plt.imshow(im_out)
plt.show()

## Para modelo 2 (5000 fotos)

In [93]:
#img = Image.open("res1.png")
#img2 = Image.open("res3.png")
print("PUNTOS REALES")
print(pts_dst)
print("PUNTOS OBTENIDOS DE LA RED")
print(pts_dst_new2)
print("-"*50)
fig = plt.figure(figsize =(25, 15))
plt.subplot(121),plt.title("1 Cuadro"),plt.imshow(img2)
plt.subplot(122),plt.title("2 Cuadros"),plt.imshow(im_out)
plt.show()

## Para modelo 3 (11000 fotos)

In [98]:
#img = Image.open("res1.png")
#img2 = Image.open("res3.png")
print("PUNTOS REALES")
print(pts_dst)
print("PUNTOS OBTENIDOS DE LA RED")
print(pts_dst_new3)
print("-"*50)
fig = plt.figure(figsize =(25, 15))
plt.subplot(121),plt.title("1 Cuadro"),plt.imshow(img2)
plt.subplot(122),plt.title("2 Cuadros"),plt.imshow(im_out)
plt.show()

# FOTO 3

## Para modelo 1 (1000 fotos)

In [133]:
#img = Image.open("res1.png")
#img2 = Image.open("res3.png")
print("PUNTOS REALES")
print(pts_dst)
print("PUNTOS OBTENIDOS DE LA RED")
print(pts_dst_new)
print("-"*50)
fig = plt.figure(figsize =(25, 15))
plt.subplot(121),plt.title("1 Cuadro"),plt.imshow(img2)
plt.subplot(122),plt.title("2 Cuadros"),plt.imshow(im_out)
plt.show()

## Para modelo 2 (5000 fotos)

In [138]:
#img = Image.open("res1.png")
#img2 = Image.open("res3.png")
print("PUNTOS REALES")
print(pts_dst)
print("PUNTOS OBTENIDOS DE LA RED")
print(pts_dst_new2)
print("-"*50)
fig = plt.figure(figsize =(25, 15))
plt.subplot(121),plt.title("1 Cuadro"),plt.imshow(img2)
plt.subplot(122),plt.title("2 Cuadros"),plt.imshow(im_out)
plt.show()

## Para modelo 3 (11000 fotos)

In [143]:
#img = Image.open("res1.png")
#img2 = Image.open("res3.png")
print("PUNTOS REALES")
print(pts_dst)
print("PUNTOS OBTENIDOS DE LA RED")
print(pts_dst_new3)
print("-"*50)
fig = plt.figure(figsize =(25, 15))
plt.subplot(121),plt.title("1 Cuadro"),plt.imshow(img2)
plt.subplot(122),plt.title("2 Cuadros"),plt.imshow(im_out)
plt.show()